In [25]:
import json
import pandas as pd 
import numpy as np 
from glob import glob
import os
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from PIL import Image
from tqdm.auto import tqdm
import roifile
import cv2


In [44]:
folder_path=glob("../../data/ddh_standard_frame_data/*")

def createDirectory(directory):
    """_summary_
        create Directory
    Args:
        directory (_type_): file_path
    """    
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")
        
def expand2square(pil_img, background_color):
    width, height = pil_img.size
    if width == height:
        return pil_img
    elif width > height:
        result = Image.new(pil_img.mode, (width, width), background_color)
        result.paste(pil_img, (0, (width - height) // 2))
        return result
    else:
        result = Image.new(pil_img.mode, (height, height), background_color)
        result.paste(pil_img, ((height - width) // 2, 0))
        return result



In [43]:
# .zip zipper and .Roi file open & save
for i in range(1,21):
    file_list=glob('../../data/pre_DDH/'+str(i)+'/seg_label/*.zip')
    output_dir = "../../data/pre_DDH/"+str(i)+"/segLabel/"
    format = "zip"
    for j in range(len(file_list)):
        createDirectory(output_dir+os.path.splitext(os.path.basename(file_list[j]))[0][:os.path.splitext(os.path.basename(file_list[j]))[0].find('_')])
        shutil.unpack_archive(file_list[j], output_dir+os.path.splitext(os.path.basename(file_list[j]))[0][:os.path.splitext(os.path.basename(file_list[j]))[0].find('_')], format)
        

NameError: name 'createDirectory' is not defined

In [102]:
# polygon to mask
roi_list=glob('../../data/pre_DDH/**/segLabel/**')
image_list=[f.replace('/segLabel', '/Standard')+'.bmp' for f in roi_list]
mask_list=[f.replace('/segLabel', '/raw_seg_mask') for f in roi_list]
for i in tqdm(range(1852,len(image_list))):
    createDirectory(mask_list[i])
    for j in range(1,4):
        try:
            roi = roifile.ImagejRoi.fromfile(roi_list[i]+'/'+str(j)+'.roi')
        except:
            try:
                roi = roifile.ImagejRoi.fromfile(roi_list[i]+'/'+str(j)+'-1.roi')
            except:
                break;
        
        image=Image.open(image_list[i])
        polygon=roi.integer_coordinates
        polygon[:,0]+=roi.left
        polygon[:,1]+=roi.top
        mask= np.zeros((image.height,image.width))
        mask=cv2.fillPoly(mask,[polygon],255)
        cv2.imwrite(mask_list[i]+'/'+str(j)+'.png',mask)


  0%|          | 0/2776 [00:00<?, ?it/s]

In [74]:
point_1[0]


array([], dtype=int64)